In [1]:
# Trying to run this on
# conda activate tf211

# Run Date: Thursday, January 12, 2023
# Run Time: 00:01:06

# Run Date: Wednesday, January 11, 2023
# Run Time: : 00:00:50


In [2]:
# Downloaded from ...
# https://www.tensorflow.org/tutorials/text/warmstart_embedding_matrix
# on ...
# Saturday, December 17, 2022

# docker run --gpus all -it -v $(realpath ~/Data):/tf/Data -p 8888:8888 -p 6006:6006 tf211:20221217

# Nice! I all runs! Tensorboard stuff does not load, but meh!
# Run Date: Saturday, December 17, 2022
# Run Time: : 00:00:50

import time
from datetime import date

startTime = time.time()
todaysDate = date.today()

In [3]:
##### Copyright 2022 The TensorFlow Authors.


# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Warm-start embedding layer matrix

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/warmstart_embedding_matrix">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/warmstart_embedding_matrix.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/warmstart_embedding_matrix.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/warmstart_embedding_matrix.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial shows how to "warm-start" training using the [`tf.keras.utils.warmstart_embedding_matrix`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/warmstart_embedding_matrix) API for text sentiment classification when changing vocabulary.

You will begin by training a simple Keras model with a base vocabulary, and then, after updating the vocabulary, continue training the model. This is referred to as "warm-start" training, for which you'll need to remap the text-embedding matrix for the new vocabulary.

## Embedding matrix

Embeddings provide a way to use an efficient, dense representation in which similar vocabulary tokens have a similar encoding. They are trainable parameters (weights learned by the model during training, in the same way a model learns weights for a dense layer). It is common to have embeddings that are 8-dimensional for small datasets, and up to 1024-dimensions when working with large datasets. A higher dimensional embedding can capture fine-grained relationships between words, but can take more data to learn.

### Vocabulary

The set of unique words is referred to as the vocabulary. To build a text model you need to choose a fixed vocabulary. Typically you you build the vocabulary from the most common words in a dataset. The vocabulary allows us to represent each piece of text by a sequence of ID's that you can lookup in the embedding matrix. Vocabulary allows us to represent each piece of text by the specific words that appear in it.

### Why warm-start an embedding matrix?

A model is trained with a set of embeddings that represents a given vocabulary. If the model needs to be updated or improved you can train to convergence significantly faster by reusing weights from a previous run. Using the embedding matrix from a previous run is more difficult. The problem is that any change to the vocabulary invalidates the word to id mapping.

The `tf.keras.utils.warmstart_embedding_matrix` solves this problem by creating an embedding matrix for a new vocabulary from an embedding martix from a base vocabulary. Where a word exists in both vocabularies the base embedding vector is copied into the correct location in the new embedding matrix. This allows you to warm-start training after any change in the size or order of the vocabulary.

## Setup

In [4]:
# !pip install --pre -U "tensorflow>2.10"  # Requires 2.11

In [5]:
import io
import numpy as np
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

2023-01-12 18:16:06.576277: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-12 18:16:07.261868: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64
2023-01-12 18:16:07.261960: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64
2023-01-12 18:16:07.261967: W tensorflow/compiler/tf2tensorrt/utils/py_utils

### Load the dataset
The tutorial uses the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/). You will train a sentiment classifier model on this dataset and in the process learn embeddings from scratch. Refer to [Loading text tutorial](https://www.tensorflow.org/tutorials/load_data/text) to learn more.  

Download the dataset using Keras file utility and review the directories.

In [6]:
# dataDirectory. Notice it is outside this repo.
dataDir = '/home/rob/Data/Documents/Github/rkaunismaa/data/'

In [7]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

# dataset = tf.keras.utils.get_file(
#     "aclImdb_v1.tar.gz", url, untar=True, cache_dir=".", cache_subdir=""
# )
dataset = tf.keras.utils.get_file(
    "aclImdb_v1.tar.gz", url, untar=True, cache_dir=dataDir, cache_subdir=""
)


In [8]:
dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")
os.listdir(dataset_dir)

['imdbEr.txt', 'imdb.vocab', 'test', 'train', 'README']

The `train/` directory has `pos` and `neg` folders with movie reviews labelled as positive and negative respectively. You will use reviews from `pos` and `neg` folders to train a binary classification model.

In [9]:
train_dir = os.path.join(dataset_dir, "train")
os.listdir(train_dir)

['urls_pos.txt',
 'urls_unsup.txt',
 'neg',
 'labeledBow.feat',
 'unsup',
 'unsupBow.feat',
 'pos',
 'urls_neg.txt']

The `train` directory also contains additional folders which should be removed before creating the training set.

In [10]:
remove_dir = os.path.join(train_dir, "unsup")
shutil.rmtree(remove_dir)

Next, create a `tf.data.Dataset` using `tf.keras.utils.text_dataset_from_directory`. You can read more about using this utility in this [text classification tutorial](https://www.tensorflow.org/tutorials/keras/text_classification). 

Use the `train` directory to create the training and validation sets with a split of 20% for validation.

In [11]:
batch_size = 1024
seed = 123
# train_ds = tf.keras.utils.text_dataset_from_directory(
#     "aclImdb/train",
#     batch_size=batch_size,
#     validation_split=0.2,
#     subset="training",
#     seed=seed,
# )
# val_ds = tf.keras.utils.text_dataset_from_directory(
#     "aclImdb/train",
#     batch_size=batch_size,
#     validation_split=0.2,
#     subset="validation",
#     seed=seed,
# )

train_dir = dataset_dir + '/train'
train_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=seed,
)
val_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=seed,
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


2023-01-12 18:16:21.213765: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 18:16:21.213988: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 18:16:21.238564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 18:16:21.238813: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 18:16:21.238963: I tensorflow/compiler/xla/stream_executo

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


### Configure the dataset for performance

You can learn more about `Dataset.cache` and `Dataset.prefetch`, as well as how to cache data to disk in the [data performance guide](https://www.tensorflow.org/guide/data_performance).

In [12]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Text preprocessing

Next, define the dataset preprocessing steps required for your sentiment classification model. Initialize a `layers.TextVectorization` layer with the desired parameters to vectorize movie reviews. You can learn more about using this layer in the [Text Classification](https://www.tensorflow.org/tutorials/keras/text_classification) tutorial.

In [13]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Make a text-only dataset (no labels) and call `Dataset.adapt` to build the
# vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


## Create a classification model

Use the [Keras Sequential API](https://www.tensorflow.org/guide/keras/sequential_model) to define the sentiment classification model. 

In [14]:
embedding_dim = 16
text_model_input = tf.keras.layers.Input(dtype=tf.string, shape=(1,))
text_embedding = Embedding(vocab_size, embedding_dim, name="embedding")

In [15]:
text_input = tf.keras.Sequential(
    [vectorize_layer, text_embedding], name="text_input"
)
classifier_head = tf.keras.Sequential(
    [GlobalAveragePooling1D(), Dense(16, activation="relu"), Dense(1)],
    name="classifier_head",
)

model = tf.keras.Sequential([text_input, classifier_head])

## Compile and train the model

In [16]:
!ls /home/rob/Data/Documents/Github/rkaunismaa/logs/TensorFlowTutorials/text

train  validation


You will use [TensorBoard](https://www.tensorflow.org/tensorboard) to visualize metrics including loss and accuracy. Create a `tf.keras.callbacks.TensorBoard`.

In [17]:
logs_dir = '/home/rob/Data/Documents/Github/rkaunismaa/logs/TensorFlowTutorials/text'
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)

Compile and train the model using the `Adam` optimizer and `BinaryCrossentropy` loss. 

In [18]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [19]:
%%time
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback],
)

Epoch 1/15


2023-01-12 18:16:26.985299: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fe4840a6d90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-01-12 18:16:26.985323: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 SUPER, Compute Capability 7.5
2023-01-12 18:16:26.988747: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-01-12 18:16:27.079613: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20/20 [==============================] - 6s 214ms/step - loss: 0.6923 - accuracy: 0.5028 - val_loss: 0.6905 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 1s 49ms/step - loss: 0.6878 - accuracy: 0.5028 - val_loss: 0.6842 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 1s 47ms/step - loss: 0.6793 - accuracy: 0.5028 - val_loss: 0.6733 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 1s 49ms/step - loss: 0.6654 - accuracy: 0.5028 - val_loss: 0.6569 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 1s 49ms/step - loss: 0.6453 - accuracy: 0.5058 - val_loss: 0.6351 - val_accuracy: 0.5048
Epoch 6/15
20/20 [==============================] - 1s 47ms/step - loss: 0.6191 - accuracy: 0.5577 - val_loss: 0.6086 - val_accuracy: 0.5770
Epoch 7/15
20/20 [==============================] - 1s 48ms/step - loss: 0.5878 - accuracy: 0.6397 - val_loss: 0.5791 - val_accuracy: 0.6458
Epoch 8/15
20/20 [=====

With this approach the model reaches a validation accuracy of around 85% 

Note: Your results may be a bit different, depending on how weights were randomly initialized before training the embedding layer. 

You can look into the model summary to learn more about each layer of the model.

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (Sequential)     (None, 100, 16)           160000    
                                                                 
 classifier_head (Sequential  (None, 1)                289       
 )                                                               
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


Visualize the model metrics in TensorBoard.

In [21]:
# docs_infra: no_execute
%load_ext tensorboard
# %tensorboard --logdir logs
%tensorboard --logdir /home/rob/Data/Documents/Github/rkaunismaa/logs/TensorFlowTutorials/text

## Vocabulary remapping

Now you're going to update the vocabulary and continue with warm-started training.

First, get the base vocabulary and embedding matrix.

In [22]:
embedding_weights_base = (
    model.get_layer("text_input").get_layer("embedding").get_weights()[0]
)
vocab_base = vectorize_layer.get_vocabulary()

Define a new vectorization layer to generate a new bigger vocabulary

In [23]:
# Vocabulary size and number of words in a sequence.
vocab_size_new = 10200
sequence_length = 100

vectorize_layer_new = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size_new,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer_new.adapt(text_ds)

# Get the new vocabulary
vocab_new = vectorize_layer_new.get_vocabulary()

In [24]:
# View the new vocabulary tokens that weren't in `vocab_base`
set(vocab_base) ^ set(vocab_new)

{'bullying',
 'bumps',
 'canvas',
 'carole',
 'chains',
 'chairman',
 'checks',
 'coarse',
 'competitive',
 'component',
 'compound',
 'confirm',
 'contemplate',
 'coping',
 'corporations',
 'costuming',
 'counterpart',
 'crop',
 'custody',
 'cyborgs',
 'daft',
 'danced',
 'daphne',
 'darkest',
 'davids',
 'december',
 'declared',
 'defence',
 'delve',
 'demonstration',
 'dense',
 'denver',
 'devilish',
 'devious',
 'dickinson',
 'digs',
 'directorwriter',
 'download',
 'effortless',
 'electricity',
 'elliot',
 'enlightenment',
 'erratic',
 'exceedingly',
 'eyeballs',
 'fearless',
 'fenton',
 'fiennes',
 'filter',
 'fireworks',
 'flipping',
 'float',
 'foggy',
 'forgivable',
 'framework',
 'fulllength',
 'funds',
 'gamut',
 'geeks',
 'glee',
 'goo',
 'gripe',
 'hardest',
 'harmony',
 'henchman',
 'heritage',
 'hg',
 'hi',
 'hightech',
 'homework',
 'houston',
 'howards',
 'hunger',
 'imho',
 'immigrants',
 'improvised',
 'impulse',
 'inch',
 'interpret',
 'intimidating',
 'iowa',
 'jaf

Generate updated embeddings using the `keras.utils.warmstart_embedding_matrix` util.

In [25]:
# Generate the updated embedding matrix
updated_embedding = tf.keras.utils.warmstart_embedding_matrix(
    base_vocabulary=vocab_base,
    new_vocabulary=vocab_new,
    base_embeddings=embedding_weights_base,
    new_embeddings_initializer="uniform",
)
# Update the model variable
updated_embedding_variable = tf.Variable(updated_embedding)

**OR**

If you have an embedding matrix which you would like to initialize the new embedding matrix with, use `keras.initializers.Constant` as new_embeddings initializer. Copy the following block to a code cell to try this out.
This would be helpful when you have a better embedding matrix initialization for new words in vocab.
```
# generate updated embedding matrix
new_embedding = np.random.rand(len(vocab_new), 16)
updated_embedding = tf.keras.utils.warmstart_embedding_matrix(
            base_vocabulary=vocab_base,
            new_vocabulary=vocab_new,
            base_embeddings=embedding_weights_base,
            new_embeddings_initializer=tf.keras.initializers.Constant(
                new_embedding
            )
        )
# update model variable
updated_embedding_variable = tf.Variable(updated_embedding)
```

Verify if the embedding matrix's shape has changed to reflect the new vocabulary.

In [26]:
updated_embedding_variable.shape

TensorShape([10200, 16])

Now that you have the updated embedding matrix, the next step is to update the layer weights.

In [27]:
text_embedding_layer_new = Embedding(
    vectorize_layer_new.vocabulary_size(), embedding_dim, name="embedding"
)
text_embedding_layer_new.build(input_shape=[None])
text_embedding_layer_new.embeddings.assign(updated_embedding)
text_input_new = tf.keras.Sequential(
    [vectorize_layer_new, text_embedding_layer_new], name="text_input_new"
)
text_input_new.summary()

# Verify the shape of updated weights
# The new weights shape should reflect the new vocabulary size
text_input_new.get_layer("embedding").get_weights()[0].shape

Model: "text_input_new"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 16)           163200    
                                                                 
Total params: 163,200
Trainable params: 163,200
Non-trainable params: 0
_________________________________________________________________


(10200, 16)

Modify the model architecture to use the new text vectorization layer.

You can also load the model from a checkpoint and update the model architecture as shown below.

In [28]:
warm_started_model = tf.keras.Sequential([text_input_new, classifier_head])
warm_started_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input_new (Sequential)  (None, 100, 16)          163200    
                                                                 
 classifier_head (Sequential  (None, 1)                289       
 )                                                               
                                                                 
Total params: 163,489
Trainable params: 163,489
Non-trainable params: 0
_________________________________________________________________


You have successfully updated the model to accept a new vocabulary. The embedding layer is updated to map old vocabulary words to old embeddings and initialize embeddings for new vocabulary words to be learnt. The learned weights of the rest of the model will remain the same. The model is warm-started to continue to train from where it left off previously.

You can now verify that the remapping worked. Get index of the vocabulary word "the" that is present both in base and new vocabulary and compare the embedding values. They should be equal.

In [29]:
# New vocab words
base_vocab_index = vectorize_layer("the")[0]
new_vocab_index = vectorize_layer_new("the")[0]
print(
    warm_started_model.get_layer("text_input_new").get_layer("embedding")(
        new_vocab_index
    )
    == embedding_weights_base[base_vocab_index]
)

tf.Tensor(
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True], shape=(16,), dtype=bool)


## Continue with warm-started training

Notice how the training is warm-started. The accuracy of first epoch is around 85%. Close to the accuracy where the previous traning ended.

In [30]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [31]:
%%time
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback],
)

Epoch 1/15
20/20 [==============================] - 4s 142ms/step - loss: 0.3477 - accuracy: 0.8515 - val_loss: 0.4045 - val_accuracy: 0.8040
Epoch 2/15
20/20 [==============================] - 1s 50ms/step - loss: 0.3333 - accuracy: 0.8594 - val_loss: 0.3978 - val_accuracy: 0.8060
Epoch 3/15
20/20 [==============================] - 1s 46ms/step - loss: 0.3208 - accuracy: 0.8635 - val_loss: 0.3922 - val_accuracy: 0.8072
Epoch 4/15
20/20 [==============================] - 1s 54ms/step - loss: 0.3094 - accuracy: 0.8679 - val_loss: 0.3877 - val_accuracy: 0.8116
Epoch 5/15
20/20 [==============================] - 1s 51ms/step - loss: 0.2988 - accuracy: 0.8745 - val_loss: 0.3841 - val_accuracy: 0.8144
Epoch 6/15
20/20 [==============================] - 1s 51ms/step - loss: 0.2889 - accuracy: 0.8789 - val_loss: 0.3813 - val_accuracy: 0.8166
Epoch 7/15
20/20 [==============================] - 1s 51ms/step - loss: 0.2796 - accuracy: 0.8834 - val_loss: 0.3792 - val_accuracy: 0.8180
Epoch 8/15
2

## Visualize warm-started training

In [32]:
# docs_infra: no_execute
%reload_ext tensorboard
# %tensorboard --logdir logs
%tensorboard --logdir /home/rob/Data/Documents/Github/rkaunismaa/logs/TensorFlowTutorials/text

Reusing TensorBoard on port 6006 (pid 223907), started 0:00:23 ago. (Use '!kill 223907' to kill it.)

<!-- <img class="tfo-display-only-on-site" src="https://tensorflow.org/tutorials/text/images/tensorboard-2.png"/> -->

## Next steps

In this tutorial you learned how to:

* Train a sentiment classification model from scratch on a small vocabulary dataset.
* Update the model architecture and warm start the embedding matrix when the vocabulary size changes.
* Continuously improve model accuracy with expanding datasets

To learn more about embeddings check out the [Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec) and [Transformer model for language understanding](https://www.tensorflow.org/text/tutorials/transformer) tutorials.

In [33]:
endTime = time.time()

elapsedTime = time.strftime("%H:%M:%S", time.gmtime(endTime - startTime))

print(todaysDate.strftime('# Run Date: %A, %B %d, %Y'))
print(f"# Run Time: {elapsedTime}")

# Run Date: Thursday, January 12, 2023
# Run Time: 00:01:06
